### 1. Import Libraries

In [1]:
import os
import sys
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### 2. Import from mlcblab

In [2]:
from mlcvlab.models.nn4 import NN4
from mlcvlab.nn.losses import l2
from mlcvlab.optim.sgd import SGD
from mlcvlab.optim.sync_sgd import sync_sgd
# TODO: Import all the necessary code from mlcvlab package as you need... 

### 3. Set Seed

In [3]:
np.random.seed(42)

### 4. Helper functions

In [4]:
def load_dataset():
      #TODO
    mnist = fetch_openml('mnist_784')
    x, y = mnist.data, mnist.target
    x = np.concatenate([np.ones((len(x), 1)), x], axis=1)#add additional column for bias terms
    # Convert the labels to integers
    y = y.astype(int)
    return x,y
def prepare_data(x, y):
       #TODO
    x = x / 255.0
    y = (y % 2 == 1).astype(int)
    y  = np.array(y).reshape(-1, 1)
    return x, y
def split_train_test(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=10000, random_state=42)
    # Subsample a smaller portion of the data for faster training
    return X_train, X_test, y_train, y_test
def minibatch(X_train,y_train,K):
    #TODO
    # Batch Size: K
    # X_train_batches, y_train_batches should be a list of lists of size K.
    n_batches = len(X_train) // K
    X_train_batches = [X_train[k*K:(k+1)*K] for k in range(n_batches)]
    y_train_batches = [y_train[k*K:(k+1)*K] for k in range(n_batches)]
    return X_train_batches, y_train_batches
def initialize_model(X_train):
    #TODO (Can use the similar approach used in HW1)
    # e.g. He Initialization for W0-W2, Xavier Initialization for W3
    # Also, initialize your model with a dropout parameter of 0.25 and use_batchnorm being true.
       #TODO
    n_input = X_train.shape[1]
    n_output = 1
    neurons = 200
    # Initialize weights using He method for layers 0-3
    W0 = np.random.randn(n_input, neurons) * np.sqrt(2 / n_input)
    W1 = np.random.randn(neurons, neurons) * np.sqrt(2 / neurons)
    W2 = np.random.randn(neurons, neurons) * np.sqrt(2 / neurons)
    W3 = np.random.randn(neurons, n_output) * np.sqrt(1 / neurons)
    print(f"Size of W0 : {W0.shape}, Size of W1 : {W1.shape}, Size of W2 : {W2.shape}, Size of W3 : {W3.shape}")
    four_layer_nn  = NN4(use_batchnorm=True, dropout_param=0.25)
    four_layer_nn.layers[0].W = W0
    four_layer_nn.layers[1].W = W1
    four_layer_nn.layers[2].W = W2
    four_layer_nn.layers[3].W = W3

    return four_layer_nn

def train_model(model, X_train_batches, y_train_batches):
    #TODO : Call async_SGD and sync_SGD to train two versions of the same model. Compare their outcomes and runtime.
    #Update both your models with final updated weights and return them
    
    #GPU Function
    #grads = sync_sgd_gpu(model, X_train_batches, y_train_batches, lr=0.1, R=100, num_gpus=2)
    
    
    #Normal Function
    grads = sync_sgd(model, X_train_batches, y_train_batches, lr=0.1, R=100)
    return grads

def test_model(model, X_test, y_test, grads):
    model.layers[0].W = grads[0]
    model.layers[1].W = grads[1]
    model.layers[2].W = grads[2]
    model.layers[3].W = grads[3]
        # Predict labels for test data
    y_pred = model.nn4(X_test)

    # Convert predicted probabilities to binary predictions
    y_pred = (y_pred >= 0.5).astype(int)

    # Compute accuracy
    accuracy =   np.mean(y_pred == y_test)

    return accuracy

### 5. Run the program

In [5]:

#load data
x, y = load_dataset()

#prepare data
x, y = prepare_data(x,y)

# split data set
X_train, X_test, y_train, y_test = split_train_test(x,y)

#initialize model
model = initialize_model(X_train)

X_train, y_train = minibatch(X_train,y_train,200)

#training model
grads =train_model(model, X_train,y_train )
print(f"Completed training, now testing...")   

#testing model
#accuracy_async = test_model(model_async, X_test, y_test)
#print(f"Completed testing model using asynchronous SGD - Accuracy : {accuracy_async}")   

accuracy_sync = test_model(model, X_test, y_test, grads)
print(f"Completed testing model using synchronous SGD - Accuracy : {accuracy_sync}") 

C:\Users\yasmi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Size of W0 : (785, 200), Size of W1 : (200, 200), Size of W2 : (200, 200), Size of W3 : (200, 1)



C:\Users\yasmi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\Users\yasmi\OneDrive\Desktop\ML in CV HW2\2023-SP-101-hw2-yam64\mlcvlab\nn\dropout.py:49: RuntimeWarning: invalid value encountered in multiply
  dz = mask * z
c:\Users\yasmi\OneDrive\Desktop\ML in CV HW2\2023-SP-101-hw2-yam64\mlcvlab\models\nn4.py:118: RuntimeWarning: invalid value encountered in multiply
  dz5 = relu_grad(z2) * dropout_grad(np.dot(dz6, self.layers[1].W.T), mask2)


Completed training, now testing...
Completed testing model using synchronous SGD - Accuracy : 0.5214
